# Graph Neural Network

* If we use graph neural networks, it can help to find relations between medical information. (ex. demographic, medical history, ...)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Layer
from sklearn.preprocessing import StandardScaler

In [ ]:
data_ori = pd.read_csv('C:/Users/user/Downloads/GNN_test.csv')
print("The shape of the original dataset is:", data_ori.shape)

In [ ]:
data_ori.columns

In [ ]:
# 빈 문자열이나 공백을 NaN으로 변환
data_ori.replace(r'^\s*$', pd.NA, regex=True, inplace=True)

In [ ]:
print(data_ori.isnull().sum())

In [ ]:
data_ori = data_ori.dropna()

In [ ]:
data_ori.shape

In [ ]:
# data_gnn = data_ori.drop(['E_No', 'HAMD_total_V1'], axis=1)
data_gnn = data_ori.drop(['E_No', 'Group'], axis=1)

In [ ]:
data_gnn.head()

In [ ]:
scaler = StandardScaler()
data_gnn_scaled = scaler.fit_transform(data_gnn)
X = data_gnn_scaled[:]

In [ ]:
X = data_gnn_scaled[:, :-1]  # 특징 행렬 (152, 10)
# y = data_gnn_scaled[:, -1]   # 레이블 (152,)

In [ ]:
data_y = data_ori.loc[:,["Group"]]

In [ ]:
y = data_y

In [ ]:
X.shape

In [ ]:
## 인접행렬(adjacency matrix) 정의
num_nodes = X.shape[0]
A = np.ones((num_nodes, num_nodes))  # 모든 노드가 연결된 완전 그래프

In [ ]:
# 그래프 합성곱 레이어 정의
class GraphConv(Layer):
    def __init__(self, units, activation=None, **kwargs): ## 합성곱 레이어의 유닛 수와 활성화 함수 초기화
        super(GraphConv, self).__init__(**kwargs) 
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.dense = Dense(units)
        
    def call(self, inputs): ## 입력받은 특징 행렬과 인접 행렬을 사용하여 그래프 합성곱 연산 수행
        features, adjacency = inputs
        output = tf.matmul(adjacency, features)
        output = self.dense(output)
        if self.activation is not None:
            output = self.activation(output)
        return output

In [ ]:
# 모델 정의
input_features = Input(shape=(num_nodes, X.shape[1]), name='X_in') ## 입력 레이어 중 특징 행렬
input_adjacency = Input(shape=(num_nodes, num_nodes), name='A_in') ## 입력 레이어 중 인접 행렬

In [ ]:
input_adjacency

In [ ]:
# # 예제 1D 데이터
# num_nodes = X.shape[0]  ## 그래프의 노드 수 
# num_features = 5  ## 그래프의 특징 수. 노드의 특징 수를 크게 하여 더 많은 정보를 제공할 수도 있음.
# X = np.random.rand(num_nodes, num_features)  # 노드 특징 행렬
# A = np.ones((num_nodes, num_nodes))          # 인접 행렬 (모든 노드가 연결된 완전 그래프)
# ## 여기서는 모든 노드가 연결된 완전 그래프를 가정.

In [ ]:
# # GraphConv 레이어 사용
# gc1 = GraphConv(32, activation='relu')([input_features, input_adjacency])
# gc2 = GraphConv(32, activation='relu')([gc1, input_adjacency])
# output = Dense(1)(gc2) ## dense 레이어를 추가해서 출력을 생성해냄.

In [ ]:
## 만약 더욱 복잡한 구조의 그래프 신경망 구현을 원한다면
gc1 = GraphConv(32, activation='relu')([input_features, input_adjacency])
gc2 = GraphConv(32, activation='relu')([gc1, input_adjacency])
gc3 = GraphConv(32, activation='relu')([gc2, input_adjacency])
gc4 = GraphConv(32, activation='relu')([gc3, input_adjacency])
output = Dense(1)(gc4)

In [ ]:
# ## 다양한 활상화 함수(activation function)을 사용하거나 batch normalization, dropout 등을 추가하고자 하는 경우.
# gc1 = GraphConv(32, activation='relu')([input_features, input_adjacency])
# gc2 = GraphConv(32, activation='relu')([gc1, input_adjacency])
# gc2 = BatchNormalization()(gc2)
# gc2 = Dropout(0.5)(gc2)
# output = Dense(1, activation='sigmoid')(gc2)

In [ ]:
# 모델 생성
model = Model(inputs=[input_features, input_adjacency], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.summary()

In [ ]:
# 데이터 형식 변환
X_train = np.expand_dims(X, axis=0)  # (1, xshape[0], xshape[1])
A_train = np.expand_dims(A, axis=0)  # (1, xshape[0], xshape[0])
y_train = np.expand_dims(y, axis=0)  # (1, xshape)

In [ ]:
X_train.shape

In [ ]:
model.fit([X_train, A_train], y_train, epochs=150, verbose=2) ## 입력데이터를 배치 차원을 추가하여 모델에 맞게 변환 및 훈련

In [ ]:
# 모델 예측 수행
predictions = model.predict([X_train, A_train])
print(predictions)

이 예제에서는 다음과 같은 단계를 수행합니다:

> 1D 입력 데이터를 준비합니다. \
> 그래프 합성곱 레이어를 직접 정의합니다. \
> Keras를 사용하여 모델을 정의하고 컴파일합니다. \
> 모델을 훈련하고 예측을 수행합니다. 

In [ ]:
y_train